<a href="https://colab.research.google.com/github/jsebastianquiroga/jsebastianquiroga/blob/main/crypto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pycoingecko

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install ccxt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install aiohttp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import ccxt
import pandas as pd
from datetime import datetime, timedelta
import pytz

# Install pytz if not already installed
# pip install pytz

exchange = ccxt.binance()
vs_currency = 'USDT'
coin_ids = ['BTC', 'SAND', 'SOL', 'BNB']
days = 365 * 4  # 4 years
timeframe = '1h'  # hourly data

all_formatted_prices = {}

# Function to generate the desired datetime objects in Bogota timezone
def generate_datetimes():
    bogota_tz = pytz.timezone('America/Bogota')
    now = datetime.now(bogota_tz)
    start_date = now - timedelta(days=days)

    datetimes = []
    current_date = start_date

    while current_date < now:
        for hour in [0, 6, 12, 18]:
            dt = current_date.replace(hour=hour, minute=0, second=0, microsecond=0)
            if dt < now:
                datetimes.append(dt)
        current_date += timedelta(days=1)

    return datetimes


desired_datetimes = generate_datetimes()

for coin_id in coin_ids:
    symbol = f'{coin_id}/{vs_currency}'
    since = exchange.parse8601((datetime.utcnow() - timedelta(days=days)).strftime('%Y-%m-%dT%H:%M:%SZ'))

    ohlcv_data = exchange.fetch_ohlcv(symbol, timeframe=timeframe, since=since)
    formatted_prices = []

    index = 0
    for dt in desired_datetimes:
        utc_dt = dt.astimezone(pytz.UTC)  # Convert Bogota time to UTC
        timestamp = int(utc_dt.timestamp()) * 1000  # Convert to milliseconds

        while index < len(ohlcv_data) - 1 and ohlcv_data[index + 1][0] <= timestamp:
            index += 1

        date = utc_dt.strftime('%Y-%m-%d %H:%M:%S')
        close_price = ohlcv_data[index][4]  # The closing price for the hour
        formatted_prices.append((date, close_price))

    all_formatted_prices[coin_id] = formatted_prices
import pandas as pd

df = pd.DataFrame()

for coin_id, prices in all_formatted_prices.items():
    temp_df = pd.DataFrame(prices, columns=['date_time', coin_id])
    temp_df['date_time'] = pd.to_datetime(temp_df['date_time'])

    if df.empty:
        df = temp_df
    else:
        df = df.merge(temp_df, on='date_time', how='outer')

df.set_index('date_time', inplace=True)
df

,BTC,SAND,SOL,BNB
date_time,,,,
2019-03-26 05:00:00,3991.59,0.065637,2.9515,16.3752
2019-03-26 11:00:00,3991.59,0.065637,2.9515,16.3752
2019-03-26 17:00:00,3991.59,0.065637,2.9515,16.3752
2019-03-26 23:00:00,3991.59,0.065637,2.9515,16.3752
2019-03-27 05:00:00,4003.48,0.065637,2.9515,16.4039
...,...,...,...,...
2023-03-23 23:00:00,5164.00,0.044745,4.6017,19.5505
2023-03-24 05:00:00,5164.00,0.044745,4.6017,19.5505
2023-03-24 11:00:00,5164.00,0.044745,4.6017,19.5505
